In [4]:
import json
import pandas as pd
import requests
import time
import datetime

In [2]:
wallet_address = 'bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx'
api_url = 'https://blockchain.info'
api_end_point = 'rawaddr'

In [5]:
url = f'{api_url}/{api_end_point}/{wallet_address}'
response = requests.get(url)
response

<Response [200]>

In [6]:
response_body = response.json()
TX_COUNT = response_body['n_tx']
TX_COUNT

1653

In [7]:
ITERATIONS = int(TX_COUNT / 100) - 1
ITERATIONS

15

In [19]:
transactions_df = None
i = 0
offset = 0
url = f'{api_url}/{api_end_point}/{wallet_address}?format=json&offset={offset}'
wallet = pd.read_json(url)
transactions_df = wallet['txs']
transactions_df.to_json(f'tx{offset}.json')
value = 30
while i <= ITERATIONS:
    offset = offset + 100
    print(offset)
    url = f'{api_url}/{api_end_point}/{wallet_address}?format=json&offset={offset}'
    print(f'requesting: {url}')
    wallet = pd.read_json(url)
    transactions_df = wallet['txs']
    transactions_df.to_json(f'tx{offset}.json')
    i = i + 1
    time.sleep(value)


100
requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx?format=json&offset=100
200
requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx?format=json&offset=200
300
requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx?format=json&offset=300
400
requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx?format=json&offset=400
500
requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx?format=json&offset=500
600
requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx?format=json&offset=600
700
requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx?format=json&offset=700
800
requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx?format=json&offset=800
900
requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6

In [8]:
i = 0
out_transactions_data = []
while i <= ITERATIONS:
    f_name = f'tx{i*100}.json'
    print(f'opeing file {f_name}')
    file = open(f_name, 'r')
    txs = json.load(file)
    address = 'bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx'
    
    for index in txs:
        tx = txs[index]
        tx_hash = tx['hash']
        tx_time = datetime.datetime.fromtimestamp(tx['time'])
        inputs = tx['inputs']
        outs = tx['out']
        is_out_tx = False
        tx_amount = 0

        for input in inputs:
            prev = input['prev_out']
            if prev['addr'] == address:
                is_out_tx= True

        if is_out_tx:
            out_tx = outs[0]
            tx_amount = float(out_tx['value']) * 0.00000001
            out_tx_address = out_tx['addr']
            # print(f'{tx_hash}, {tx_time}, {address}, {tx_amount}, {out_tx_address}')
            out_transactions_data.append([tx_hash, tx_time, address, tx_amount, out_tx_address])

    i = i + 1

opeing file tx0.json
opeing file tx100.json
opeing file tx200.json
opeing file tx300.json
opeing file tx400.json
opeing file tx500.json
opeing file tx600.json
opeing file tx700.json
opeing file tx800.json
opeing file tx900.json
opeing file tx1000.json
opeing file tx1100.json
opeing file tx1200.json
opeing file tx1300.json
opeing file tx1400.json
opeing file tx1500.json


In [9]:
result = pd.DataFrame(out_transactions_data, columns=['tx_hash', 'tx_date_time', 'from_addr', 'amount', 'to_addr'])
result

,tx_hash,tx_date_time,from_addr,amount,to_addr
0,53392049d344b4f016d3560dece5765d43e42bdfb181ff...,2021-12-25 05:17:20,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.001008,bc1qlx09ds46r0l54t0aawn79x5t9mef2sj9dk2ce7
1,52b4599f00e6c75d6c454f3ae6c29097dbe3eba8a9d248...,2021-12-12 15:51:53,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.000443,3DscuRBWfD927YKHnd17pzAdeRSXApaeM6
2,91a51216ce148b2a7636db67e1725ea53d1b133947e11b...,2021-12-08 07:27:00,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.009322,bc1qs3uxa3686702x4mpx2tt2y99s65n706yun5ey3
3,f8293c5e6ed4febe48294eaebcce91a0721cd2680e9118...,2021-12-08 07:27:00,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.009322,bc1qs3uxa3686702x4mpx2tt2y99s65n706yun5ey3
4,dbddbf8e66ab74dbe53164fa77d9cb770dbaddf4e92e9b...,2021-12-08 07:27:00,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.009322,bc1qs3uxa3686702x4mpx2tt2y99s65n706yun5ey3
...,...,...,...,...,...
465,87d876acff810967d9bc8caeb7aa3be5de776cc0e3cb6a...,2021-01-05 06:56:11,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.047795,bc1q5j5ddxy08jxdg9zcj7gdd3x34dkq9c79cnje46
466,321c1d4099f67f79bfb70c5b3d2ca7a05e4645f3148db7...,2021-01-04 20:53:04,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.005987,37ryD6EVu7EnKqGRL1SGVNcyS13xngdEnB
467,ad929420eb2748a785a76ae555e6e59c72641d0bc56222...,2021-01-04 20:53:04,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.003602,1GBvTTuLgvQ1DxbbiRD4BDyumWd5jkwQy9
468,ab6433eebec9f195437b8489ca0e31ab069ffd34b943f7...,2021-01-04 20:53:04,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.001200,3DvuaGTyug586Rbuk38PRCXpoA3XEqVs5B


In [11]:
result['amount'].sum()

9.136901310000002

In [19]:
result.to_csv('bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx_out_trx_data.csv', sep=';', index=False)

9.136901310000002